Objective: Take local CSV file containing all bike trips for June and just extract the unique station locations for integration with map.

In [69]:
import pandas as pd

df = pd.read_csv('JC-202305-citibike-tripdata.csv')
column_list = df.columns.tolist()
df = df.drop(columns=['ride_id', 'rideable_type', 'started_at', 'ended_at', 'started_at', 'ended_at', 'member_casual'])

#re-arranging for legibility
columns = df.columns.tolist()
columns.insert(5, columns.pop(2))
columns.insert(5, columns.pop(2))

df = df[columns]
df = df.dropna()

print(df)

                                 start_station_name start_station_id  \
0                                    Pershing Field            JC024   
1                                    Pershing Field            JC024   
2                                    Pershing Field            JC024   
3                                    Pershing Field            JC024   
4                                    Pershing Field            JC024   
...                                             ...              ...   
95997  South Waterfront Walkway - Sinatra Dr & 1 St            HB103   
95998  South Waterfront Walkway - Sinatra Dr & 1 St            HB103   
95999  South Waterfront Walkway - Sinatra Dr & 1 St            HB103   
96000                                Columbus Drive            JC014   
96001  South Waterfront Walkway - Sinatra Dr & 1 St            HB103   

       start_lat  start_lng       end_station_name end_station_id    end_lat  \
0      40.742675 -74.051806           Newport PATH     

In [70]:
station_coordinates = {}

# Iterate through the rows of the DataFrame
for index, row in df.iterrows():
    station_id = row['start_station_id']
    lat = row['start_lat']
    lng = row['start_lng']
    name = row['start_station_name']
    
    # Check if the station_id is already in the dictionary
    if station_id not in station_coordinates:
        station_coordinates[station_id] = (lng, lat, name)

for index, row in df.iterrows():
    station_id = row['end_station_id']
    lat = row['end_lat']
    lng = row['end_lng']
    name = row['end_station_name']

    # Check if the station_id is already in the dictionary
    if station_id not in station_coordinates:
        station_coordinates[station_id] = (lng, lat, name)

# Display the dictionary
print(station_coordinates)



{'JC024': (-74.051805854, 40.742674708, 'Pershing Field'), 'JC019': (-74.0575736, 40.7311689, 'Hilltop'), 'JC080': (-74.0572714805603, 40.74590996631558, 'Leonard Gordon Park'), 'JC020': (-74.0641943, 40.7236589, 'Baldwin at Montgomery'), 'JC022': (-74.0524783, 40.7376037, 'Oakland Ave'), 'HB102': (-74.02898562, 40.736364603, 'Hoboken Terminal - River St & Hudson Pl'), 'HB602': (-74.02698867022991, 40.74313282710551, 'Stevens - River Ter & 6 St'), 'HB506': (-74.031502366, 40.753829479, 'Grand St & 14 St'), 'HB407': (-74.03690412640572, 40.73981372311843, 'Adams St & 2 St'), 'HB105': (-74.03097, 40.73736, 'City Hall - Washington St & 1 St'), 'HB103': (-74.02778059244156, 40.73698221818716, 'South Waterfront Walkway - Sinatra Dr & 1 St'), 'JC059': (-74.040608048, 40.748777151, 'Heights Elevator'), '4317.05': (-74.046973348, 40.735353589, 'Bushwick Ave & Furman Ave'), 'HB409': (-74.035703778, 40.737485647, 'Clinton St & Newark St'), 'HB302': (-74.034407258, 40.744283915, '6 St & Grand St'

In [71]:
invalid_pairs = [(key, value) for key, value in station_coordinates.items() if not -80 <= value[0] <= -70]

# Print the result
if len(invalid_pairs) == 0:
    print("All values at index 0 are between -80 and -70.")
else:
    print("The following key-value pairs have values at index 0 outside the range -80 to -70:")
    for key, value in invalid_pairs:
        print(key, value)

All values at index 0 are between -80 and -70.


In [72]:
import json

features = []
for key, value in station_coordinates.items():
    lat, lon, name = value
    if -90 <= lat <= 90 and -180 <= lon <= 180:
        point = {
            'type': 'Point',
            'coordinates': [lat, lon]
        }
        feature = {
            'type': 'Feature',
            'properties': {'name': name},
            'geometry': point
        }
        features.append(feature)

feature_collection = {
    'type': 'FeatureCollection',
    'features': features
}

# Convert the FeatureCollection to GeoJSON string
geojson_str = json.dumps(feature_collection)

# Save the GeoJSON to a file
with open('bikes.geojson', 'w') as file:
    file.write(geojson_str)

print(geojson_str)


{"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {"name": "Pershing Field"}, "geometry": {"type": "Point", "coordinates": [-74.051805854, 40.742674708]}}, {"type": "Feature", "properties": {"name": "Hilltop"}, "geometry": {"type": "Point", "coordinates": [-74.0575736, 40.7311689]}}, {"type": "Feature", "properties": {"name": "Leonard Gordon Park"}, "geometry": {"type": "Point", "coordinates": [-74.0572714805603, 40.74590996631558]}}, {"type": "Feature", "properties": {"name": "Baldwin at Montgomery"}, "geometry": {"type": "Point", "coordinates": [-74.0641943, 40.7236589]}}, {"type": "Feature", "properties": {"name": "Oakland Ave"}, "geometry": {"type": "Point", "coordinates": [-74.0524783, 40.7376037]}}, {"type": "Feature", "properties": {"name": "Hoboken Terminal - River St & Hudson Pl"}, "geometry": {"type": "Point", "coordinates": [-74.02898562, 40.736364603]}}, {"type": "Feature", "properties": {"name": "Stevens - River Ter & 6 St"}, "geometry": {"type":